In [22]:
#Loading Libraries
# %conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
# %pip install -U adapter-transformers
# %conda install -y -c conda-forge tensorboard
# %pip install optuna
# %pip install tqdm
from tqdm.notebook import tqdm

In [1]:
# Loading dataset
from datasets import load_dataset

scierc_name = 'nsusemiehl/SciERC'
scierc_dataset = load_dataset(scierc_name)
print(scierc_dataset.num_rows)

Using custom data configuration nsusemiehl--SciERC-f57c64a52b9c80c0
Reusing dataset json (C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
100%|██████████| 3/3 [00:00<00:00, 999.12it/s]

{'train': 3219, 'test': 974, 'validation': 455}


In [2]:
scierc_dataset['train'][255]

{'text': 'We present two [[ methods ]] for capturing << nonstationary chaos >> , then present a few examples including biological signals , ocean waves and traffic flow .',
 'label': 'USED-FOR',
 'metadata': [3, 3, 6, 7]}

This block creates dataset for pretraining

In [3]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# Tokenize the set for the transformer
def encode_batch_pretraining(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

# Encode the input data
# NOTE: num_proc does not seem to work, for some reason it can't find the tokenizer
scierc_dataset_pretraining = scierc_dataset.map(encode_batch_pretraining, 
                                    batched=True, 
                                    remove_columns=scierc_dataset['train'].column_names, 
                                    )

# We make the labels the same as the input as this is language learning 
def add_labels(examples):
  examples["labels"] = examples["input_ids"].copy()
  return examples
  
scierc_dataset_pretraining = scierc_dataset_pretraining.map(add_labels, batched=True)
scierc_dataset_pretraining.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-74c87c445e04c867.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-876e0d37d233bb91.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-ecd8f07822814f57.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-b972bbb5f00d5d6e.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl-

In [4]:
# Collater adds padding in the form of EOS tokens, makes data augmentations of random masking ('mlm_probability)
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Here we are creating the dataset for task finetuning

In [5]:
# Finding the number of labels
import numpy as np
labels = np.unique(np.array(scierc_dataset['train']['label']))
num_of_labels = labels.size

print(labels)
print(num_of_labels)

['COMPARE' 'CONJUNCTION' 'EVALUATE-FOR' 'FEATURE-OF' 'HYPONYM-OF'
 'PART-OF' 'USED-FOR']
7


In [6]:
# encoding the labels
def encode_labels(dataset):
    for i in range(num_of_labels):
        if dataset['label'] == labels[i]:
            dataset['label'] = i
    return dataset

scierc_dataset = scierc_dataset.map(encode_labels)
scierc_dataset['train'][0]

Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-3fa4decd4606a523.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-4d2e52dbe4cdbad6.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-214c6724dd02783d.arrow


{'text': 'The agreement in question involves number in [[ nouns ]] and << reflexive pronouns >> and is syntactic rather than semantic in nature because grammatical number in English , like grammatical gender in languages such as French , is partly arbitrary .',
 'label': 1,
 'metadata': [7, 7, 9, 10]}

In [7]:
def encode_batch_finetuning(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
scierc_dataset_finetuning = scierc_dataset.map(encode_batch_finetuning, batched=True)
# The transformers model expects the target class column to be named "labels"
scierc_dataset_finetuning = scierc_dataset_finetuning.rename_column("label", 'labels')
# Transform to pytorch tensors and only output the required columns
scierc_dataset_finetuning.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-6c719cda162c2a70.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-a2e89e74a8a70442.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-1e077601566683c7.arrow


# Model Creation

In [8]:
from transformers import RobertaConfig
from transformers import RobertaAdapterModel

def model_init(adapter_name = 'default_adapter', 
               num_lables = 0, 
               pretraining = False,
               load_adapter = False,
               adapter_dir = 'path'):
    """Creates a new roBERTa model with the given name for its adapter.

    Args:
        adapter_name (str): The name of the adapter to load/create. Defaults to 'default_adapter'.
        num_lables (int, optional): The number of labels for classification task. Defaults to 0.
        pretraining (bool, optional): Whether to create a model for pretraining or classification. Defaults to False.
        load_adapter (bool, optional): Whether to load an adapter with the adapter_name given or create a new one. Defaults to False.
        adapter_dir (str, optional): Directory to load the adapter. If load_adapter you need to specify this.  Defaults to 'path'.

    Returns:
        RobertaAdapterModel: A roBERTA model with an adapter added to it.
    """
    
    if pretraining:
        config = RobertaConfig.from_pretrained(
            "roberta-base",
            # num_labels=num_of_labels,*-8536.22.03
        )
        model = RobertaAdapterModel.from_pretrained(
            "roberta-base",
            config=config,
        )
        if load_adapter:
            # Add new adapter
            model.load_adapter(adapter_dir)

        else:
            # Add new adapter
            model.add_adapter(adapter_name)
            
        # Add a matching classification head
        model.add_masked_lm_head(adapter_name)
            
    else:
        config = RobertaConfig.from_pretrained(
            "roberta-base",
            num_labels=num_lables,
        )
        model = RobertaAdapterModel.from_pretrained(
            "roberta-base",
            config=config,
        )
        
        if load_adapter:
            # Add new adapter
            model.load_adapter(adapter_dir)

        else:
            # Add new adapter
            model.add_adapter(adapter_name)
            
        # Add a matching classification head
        model.add_classification_head(
                adapter_name,
                num_labels=num_lables,
                id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
                        3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'},
                overwrite_ok = True)
            
    # Activate the adapter
    model.train_adapter(adapter_name)    
     
    return model

Pretraining Block

In [25]:
from transformers import TrainingArguments, AdapterTrainer
from datasets import load_metric
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback

import json

def pretraining_loop(num_models, training_args, dataset, 
                     data_collator, adapter_name, 
                    #  DAPT_n_TAPT, TAPT_dataset
                     ):
    """The Loop for running num_models number of models to determine run to run variance. Will run the model 
        and evaluate.

    Args:
        num_models (int): Number of models to loop through
        training_args (transformers.TrainingArguments): The arguments to pass to the trainer
        dataset (dataset): The dataset to train on
        data_collator (data_collator): The data collator for the trainer to use
        adapter_name (str): Name of the adapter to create
    """

    for i in range(num_models):
        adapter = f"{adapter_name}_{i}"
        model = model_init(adapter_name = adapter, pretraining=True)
        
        writer = SummaryWriter(log_dir= f'runs/{adapter}')
        writer = TensorBoardCallback(writer)

        trainer = AdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            data_collator=data_collator,  
            callbacks=[writer] 
        )
        
        trainer.train()
        
        f = open(f"{training_args.output_dir}/evaulations.txt", "a")
        f.write(adapter)
        f.write(json.dumps(trainer.evaluate(dataset['test'])))
        f.write('\n')
        f.close()
        
        model.save_all_adapters(training_args.output_dir, with_head=False)
        # model.save_pretrained(f"{adapter_name}")
        
        # if DAPT_n_TAPT:
        #     trainer = AdapterTrainer(
        #         model=model,
        #         args=training_args,
        #         train_dataset=TAPT_dataset["train"],
        #         eval_dataset=TAPT_dataset["validation"],
        #         data_collator=data_collator,  
        #         callbacks=[writer] 
        #     )
            
        #     trainer.train()
        
        #     f = open("DAPT_TAPT_evaulations.txt", "a")
        #     f.write(adapter_name)
        #     f.write(trainer.evaluate(TAPT_dataset['test']))
        #     f.write('\n')
        #     f.close()
            
        #     model.save_pretrained(f"{adapter_name}_DAPT_TAPT")

DAPT Training

In [10]:
training_args = TrainingArguments(
    learning_rate=5e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    output_dir="./training_output/pretraining/DAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100,
    gradient_accumulation_steps = 64,
    warmup_ratio = 0.06,
    weight_decay=0.01,
    adam_epsilon = 1e-6,
)

In [ ]:
pretraining_loop(num_models = 5, 
                 training_args = training_args, 
                #  dataset = DAPT_dataset, TODO: Need to add DAPT training set
                 data_collator = data_collator, 
                 adapter_name = "DAPT_sci-erc")

DAPT+TAPT Training

TAPT Training

In [10]:
training_args = TrainingArguments(
    learning_rate=0.0001,
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/pretraining/TAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100,
    gradient_accumulation_steps = 8,
    warmup_ratio = 0.06,
    # load_best_model_at_end = True,
    weight_decay=0.01,
    adam_epsilon = 1e-6,
)

In [11]:
pretraining_loop(num_models = 1, 
                 training_args = training_args, 
                 dataset = scierc_dataset_pretraining, 
                 data_collator = data_collator, 
                 adapter_name = "TAPT_sci-erc")

 98%|█████████▊| 1180/1200 [1:08:51<01:07,  3.37s/it]***** Running Evaluation *****
  Num examples = 455
  Batch size = 32


{'loss': 5.2076, 'learning_rate': 1.7730496453900712e-06, 'epoch': 98.32}


                                                     
 98%|█████████▊| 1180/1200 [1:08:54<01:07,  3.37s/it]

{'eval_loss': 5.05110502243042, 'eval_runtime': 2.7725, 'eval_samples_per_second': 164.111, 'eval_steps_per_second': 5.41, 'epoch': 98.32}


 99%|█████████▉| 1190/1200 [1:09:27<00:35,  3.56s/it]***** Running Evaluation *****
  Num examples = 455
  Batch size = 32


...

                                                     
100%|██████████| 1200/1200 [1:10:04<00:00,  3.20s/it]Saving model checkpoint to ./training_output/pretraining/TAPT\checkpoint-1200
Configuration saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\adapter_config.json
Module weights saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\pytorch_adapter.bin
Configuration saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\head_config.json


{'eval_loss': 5.0362467765808105, 'eval_runtime': 2.6884, 'eval_samples_per_second': 169.243, 'eval_steps_per_second': 5.579, 'epoch': 99.95}


Module weights saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\pytorch_model_head.bin
Configuration saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\pytorch_model_head.bin
Configuration saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/pretraining/TAPT\checkpoint-1200\TAPT_sci-erc_0\pytorch_model_head.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1200/1200 [1:10:05<00:00,  3.50s/it]
***** Running Evaluation *****
  Num examples = 974
  Batch size = 32


{'train_runtime': 4205.2974, 'train_samples_per_second': 76.546, 'train_steps_per_second': 0.285, 'train_loss': 6.185284856160481, 'epoch': 99.95}


100%|██████████| 31/31 [00:05<00:00,  5.48it/s]
Configuration saved in ./training_output/pretraining/TAPT\TAPT_sci-erc_0\adapter_config.json
Module weights saved in ./training_output/pretraining/TAPT\TAPT_sci-erc_0\pytorch_adapter.bin
Configuration saved in ./training_output/pretraining/TAPT\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/pretraining/TAPT\TAPT_sci-erc_0\pytorch_model_head.bin


Fine Tuning Models

In [12]:
from datasets import load_metric
metric = load_metric('f1')

def compute_metric(EvalPrediction):
  
  logits, labels = EvalPrediction
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels, average= 'macro')

In [13]:
def finetuning_loop(num_models, training_args, dataset, adapter_name, num_labels, load_adapter = False, adapter_dir = 'Path'):

    for i in range(num_models):
        adapter = f"{adapter_name}_{i}"
        model = model_init(adapter_name = adapter, num_lables = num_labels, pretraining=False, load_adapter = load_adapter, adapter_dir = f"{adapter_dir}/{adapter}")
        
        writer = SummaryWriter(log_dir= f'runs/{adapter}')
        writer = TensorBoardCallback(writer)

        trainer = AdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            callbacks=[writer],
            compute_metrics = compute_metric 
        )
        
        trainer.train()
        
        f = open(f"{training_args.output_dir}/evaulations.txt", "a")
        f.write(adapter)
        f.write(json.dumps(trainer.evaluate(dataset['test'])))
        f.write('\n')
        f.close()
        
        # model.save_pretrained(f"{adapter_name}")
        model.save_all_adapters(training_args.output_dir)
        
        trainer.remove_callback(writer)

DAPT Finetuning

In [12]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    output_dir="./training_output/finetuning/DAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy = 'epoch',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
finetuning_loop(num_models = 5, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "DAPT_sci-erc",
                 load_adapter = True)

DAPT+TAPT Finetuning

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    output_dir="./training_output/finetuning/DAPT_TAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy = 'epoch',
    # load_best_model_at_end = True,
    save_steps = 100
)

In [ ]:
finetuning_loop(num_models = 5, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "DAPT_TAPT_sci-erc",
                 load_adapter = True,
                 adapter_dir = "./training_output/pretraining/DAPT_TAPT",
                 num_labels = num_of_labels)

TAPT Finetuning

In [23]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    output_dir="./training_output/finetuning/TAPT",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy = 'epoch',
    # load_best_model_at_end = True,
    save_steps = 100,
    lr_scheduler_type = 'constant',
    log_level  = 'error'
    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
finetuning_loop(num_models = 1, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "TAPT_sci-erc",
                 load_adapter = True,
                 adapter_dir = "./training_output/pretraining/TAPT",
                 num_labels = num_of_labels)

 73%|███████▎  | 7400/10100 [13:56<04:32,  9.90it/s]Saving model checkpoint to ./training_output/finetuning/TAPT\checkpoint-7400
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\adapter_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\pytorch_adapter.bin
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\pytorch_model_head.bin
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\pytorch_model_head.bin
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-7400\TAPT_sci-erc_0\pytorch_model_head.bin
 73

{'loss': 0.3206, 'learning_rate': 2e-05, 'epoch': 36.63}


 74%|███████▎  | 7426/10100 [13:58<04:35,  9.71it/s]

{'loss': 0.2375, 'learning_rate': 2e-05, 'epoch': 36.76}


 74%|███████▍  | 7452/10100 [14:01<04:30,  9.80it/s]

...

100%|██████████| 10100/10100 [19:06<00:00,  9.36it/s]Saving model checkpoint to ./training_output/finetuning/TAPT\checkpoint-10100
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\adapter_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\pytorch_adapter.bin
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\pytorch_model_head.bin
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\pytorch_model_head.bin
Configuration saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\checkpoint-10100\TAPT_sci-erc_0\pytorch_model_he

{'loss': 0.0939, 'learning_rate': 2e-05, 'epoch': 50.0}


                                                     
100%|██████████| 10100/10100 [19:08<00:00,  9.36it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 10100/10100 [19:08<00:00,  8.80it/s]
***** Running Evaluation *****
  Num examples = 974
  Batch size = 16


{'eval_loss': 0.5171557664871216, 'eval_f1': 0.8222261813681893, 'eval_runtime': 1.5634, 'eval_samples_per_second': 291.028, 'eval_steps_per_second': 18.549, 'epoch': 50.0}
{'train_runtime': 1148.1544, 'train_samples_per_second': 140.181, 'train_steps_per_second': 8.797, 'train_loss': 0.5226877824622805, 'epoch': 50.0}


100%|██████████| 61/61 [00:03<00:00, 18.78it/s]
Configuration saved in ./training_output/finetuning/TAPT\TAPT_sci-erc_0\adapter_config.json
Module weights saved in ./training_output/finetuning/TAPT\TAPT_sci-erc_0\pytorch_adapter.bin
Configuration saved in ./training_output/finetuning/TAPT\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\TAPT_sci-erc_0\pytorch_model_head.bin
Configuration saved in ./training_output/finetuning/TAPT\TAPT_sci-erc_0\head_config.json
Module weights saved in ./training_output/finetuning/TAPT\TAPT_sci-erc_0\pytorch_model_head.bin


Only Finetuning

In [ ]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    output_dir="./training_output/finetuning/No_Pretrain",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
    save_steps = 100,
    lr_scheduler_type = 'constant',
)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
finetuning_loop(num_models = 1, 
                 training_args = training_args, 
                 dataset = scierc_dataset_finetuning,  
                 adapter_name = "sci-erc",
                 load_adapter = False,
                 num_labels = num_of_labels)